In [1]:
!pip install pinecone-client pandas numpy scipy transformers torch scikit-learn
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
import numpy as np
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, pipeline
import torch
import json
import io
from scipy.stats import median_abs_deviation
from google.colab import files

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key='4cf09f70-42a8-4fab-a374-d1fcddf313e7')
#pc = Pinecone(api_key='ff8921a2-ec42-4332-a2bb-0e0e6a3cf87f')
index_name = "temp2train"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [81]:
print("Please upload your training files.")
uploaded_training = files.upload()

# Load and concatenate training data
dataframes = [pd.read_csv(io.BytesIO(uploaded_training[filename])) for filename in uploaded_training.keys()]
training_data = pd.concat(dataframes, ignore_index=True)

# Verify the training files
print(training_data.head())
print(training_data.shape)
print(training_data.columns)

Please upload your training files.


Saving transformed_transposed_issue1_metrics.csv to transformed_transposed_issue1_metrics (4).csv
Saving transformed_transposed_issue0_metrics.csv to transformed_transposed_issue0_metrics (4).csv
           microservice            timestamp  availability_Average  \
0  169.254.170.2_remote  2023-04-22 06:30:00                   0.0   
1  169.254.170.2_remote  2023-04-22 06:35:00                   0.0   
2  169.254.170.2_remote  2023-04-22 06:40:00                   0.0   
3  169.254.170.2_remote  2023-04-22 06:45:00                 100.0   
4  169.254.170.2_remote  2023-04-22 06:50:00                 100.0   

   latency_Average  latency_p50  latency_p90  latency_p95  latency_p99  \
0         0.000000     0.000000     0.000000     0.000000     0.000000   
1         0.000000     0.000000     0.000000     0.000000     0.000000   
2         0.000000     0.000000     0.000000     0.000000     0.000000   
3         0.000574     0.000574     0.000574     0.000574     0.000574   
4         0.0

In [52]:
print("Please upload your test file.")
uploaded_test = files.upload()

# Load test data
test_file = list(uploaded_test.keys())[0]
test_data = pd.read_csv(io.BytesIO(uploaded_test[test_file]))

# Verify and print details of the test file
print("Test file:")
print(f"Filename: {test_file}")
print(f"Shape: {test_data.shape}")
print(f"Columns: {test_data.columns.tolist()}")
print(f"Head:\n{test_data.head()}\n")

Please upload your test file.


Saving transformed_transposed_issue5_metrics.csv to transformed_transposed_issue5_metrics (1).csv
Test file:
Filename: transformed_transposed_issue5_metrics (1).csv
Shape: (205, 9)
Columns: ['microservice', 'timestamp', 'availability_Average', 'latency_Average', 'latency_p50', 'latency_p90', 'latency_p95', 'latency_p99', 'requests_Sum']
Head:
           microservice            timestamp  availability_Average  \
0  169.254.170.2_remote  2023-04-22 12:40:00                   0.0   
1  169.254.170.2_remote  2023-04-22 12:45:00                 100.0   
2  169.254.170.2_remote  2023-04-22 12:50:00                 100.0   
3  169.254.170.2_remote  2023-04-22 12:55:00                   0.0   
4  169.254.170.2_remote  2023-04-22 13:00:00                   0.0   

   latency_Average  latency_p50  latency_p90  latency_p95  latency_p99  \
0         0.000000     0.000000     0.000000     0.000000     0.000000   
1         0.000658     0.000658     0.000658     0.000658     0.000658   
2         0.

In [5]:
# Upload JSON file
uploaded_json = files.upload()
print("Uploaded JSON file:")
for filename in uploaded_json.keys():
    print(filename)

# Load JSON data
json_filename = list(uploaded_json.keys())[0]
with open(json_filename, 'r') as f:
    dependencies_data = json.load(f)

# Print JSON structure
print(json.dumps(dependencies_data, indent=2))

# Process service dependency information
service_info = {service['name']: {'dependencies': service['dependencies'], 'dependents': service['dependents']} for service in dependencies_data['services']}

# Print service dependencies and dependents
for service, info in service_info.items():
    print(f"Service: {service}")
    print(f"  Dependencies: {', '.join(info['dependencies'])}")
    print(f"  Dependents: {', '.join(info['dependents'])}")
    print()


Saving service_dependency_information.json to service_dependency_information.json
Uploaded JSON file:
service_dependency_information.json
{
  "services": [
    {
      "name": "169.254.170.2_remote",
      "dependencies": [],
      "dependents": [
        "PetSearch_AWS::ECS::Fargate"
      ]
    },
    {
      "name": "AWSSimpleSystemsManagement_AWS::AWSSimpleSystemsManagement",
      "dependencies": [],
      "dependents": [
        "PetSearch_AWS::ECS::Fargate"
      ]
    },
    {
      "name": "DynamoDB_AWS::DynamoDB",
      "dependencies": [],
      "dependents": [
        "lambda_step_readDDB"
      ]
    },
    {
      "name": "Evidently_AWS::Evidently",
      "dependencies": [],
      "dependents": [
        "lambda_step_priceGreaterThan55",
        "lambda_step_priceLessThan55"
      ]
    },
    {
      "name": "PGSQL Query_remote",
      "dependencies": [],
      "dependents": [
        "petlistadoptions_AWS::ECS::Fargate"
      ]
    },
    {
      "name": "PetAdoptionStat

In [6]:
# Load tokenizer and model for embedding generation
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-en-v1.5')
model.to('cuda')  # Use GPU

# Define function to convert text to embedding
def text_to_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to('cuda') for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output[0].cpu().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [83]:
# Upload data to Pinecone
index=pc.Index('temp2train')
for i, row in training_data.iterrows():
    combined_text = f"Microservice: {row['microservice']}, Timestamp: {row['timestamp']}, Availability: {row['availability_Average']}, Latency Average: {row['latency_Average']}, Latency p50: {row['latency_p50']}, Latency p90: {row['latency_p90']}, Latency p95: {row['latency_p95']}, Latency p99: {row['latency_p99']}, Requests Sum: {row['requests_Sum']}"
    vec = text_to_embedding(combined_text)
    metadata = {
        'microservice': row['microservice'],
        'timestamp': str(row['timestamp']),
        'availability_Average': row['availability_Average'],
        'latency_Average': row['latency_Average'],
        'latency_p50': row['latency_p50'],
        'latency_p90': row['latency_p90'],
        'latency_p95': row['latency_p95'],
        'latency_p99': row['latency_p99'],
        'requests_Sum': row['requests_Sum'],
        #'issue_injected' : row['issue_injected'],
        #'path': row['path'],
        #'textual_representation': row['textual_representation']
    }

    index.upsert(vectors=[(str(i), vec, metadata)])

In [53]:
def calculate_mad_scores(dataframe):
    """Calculate MAD scores."""
    metrics = ['availability_Average', 'latency_Average', 'latency_p50', 'latency_p90', 'latency_p95', 'latency_p99', 'requests_Sum']
    mad_columns = [m + '_MAD' for m in metrics]
    for m in metrics:
        dataframe[m + '_MAD'] = dataframe.groupby('microservice')[m].transform(lambda x: median_abs_deviation(x, scale='normal'))
    dataframe['Max_MAD_Score'] = dataframe[mad_columns].max(axis=1)
    dataframe['Metric_With_Max_MAD'] = dataframe[mad_columns].idxmax(axis=1).str.replace('_MAD', '')
    return dataframe

# Calculate MAD scores for test data
mad_scores_df = calculate_mad_scores(test_data)

# Get the data point with the highest MAD score
max_mad_score_row = mad_scores_df.loc[mad_scores_df['Max_MAD_Score'].idxmax()]

print("Data point with the highest MAD score:")
print(max_mad_score_row)

Data point with the highest MAD score:
microservice                petInfo_AWS::DynamoDB::Table
timestamp                            2023-04-22 12:40:00
availability_Average                               100.0
latency_Average                                 0.013563
latency_p50                                     0.003781
latency_p90                                     0.031199
latency_p95                                     0.058845
latency_p99                                     0.136555
requests_Sum                                       889.0
availability_Average_MAD                             0.0
latency_Average_MAD                             0.000031
latency_p50_MAD                                 0.000017
latency_p90_MAD                                  0.00089
latency_p95_MAD                                 0.001532
latency_p99_MAD                                 0.056319
requests_Sum_MAD                              108.229962
Max_MAD_Score                                 108

In [55]:
index=pc.Index('temp2train')
def query_pinecone(index, query_embedding):
    query_embedding = query_embedding.tolist()
    response = index.query(
        vector=query_embedding,
        top_k=5,
        include_metadata=True
    )
    return response

# Embed the highest anomaly data and query Pinecone
def create_combined_text(row):
    return (
        f"Microservice: {row['microservice']}, Timestamp: {row['timestamp']}, "
        f"Availability: {row['availability_Average']}, Latency Average: {row['latency_Average']}, "
        f"Latency p50: {row['latency_p50']}, Latency p90: {row['latency_p90']}, "
        f"Latency p95: {row['latency_p95']}, Latency p99: {row['latency_p99']}, Requests Sum: {row['requests_Sum']}"
    )

combined_text = create_combined_text(max_mad_score_row)
embedding = text_to_embedding(combined_text)

# Debug: Print embedding to ensure validity
print("Embedding:")
print(embedding)

# Query Pinecone
response = query_pinecone(index, embedding)

# Verify the queried matches and retrieve metadata
print("Queried matches:")
for match in response['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}, Metadata: {match['metadata']}")
    metadata = match['metadata']

Embedding:
[-0.93062353 -0.66319036 -0.18178828 ...  0.36778724  0.9818483
 -0.92594963]
Queried matches:
ID: 174, Score: 0.999994338, Metadata: {'availability_Average': 100.0, 'latency_Average': 0.0135342812254714, 'latency_p50': 0.0038708019828941, 'latency_p90': 0.0280731867769307, 'latency_p95': 0.0459056850322154, 'latency_p99': 0.3736057591813309, 'microservice': 'petInfo_AWS::DynamoDB::Table', 'requests_Sum': 999.0, 'timestamp': '2023-04-22 06:50:00'}
ID: 376, Score: 0.999859333, Metadata: {'availability_Average': 100.0, 'latency_Average': 0.0144206176749589, 'latency_p50': 0.0039494934835474, 'latency_p90': 0.0324727479583365, 'latency_p95': 0.0509933338109614, 'latency_p99': 0.3607461879284954, 'microservice': 'petInfo_AWS::DynamoDB::Table', 'requests_Sum': 658.0, 'timestamp': '2023-04-21 05:25:00'}
ID: 375, Score: 0.999834359, Metadata: {'availability_Average': 100.0, 'latency_Average': 0.012674660924489, 'latency_p50': 0.0037912315747268, 'latency_p90': 0.0287159030135047, '

In [9]:
# Load the Mistral model for response generation
mistral_tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-2-Pro-Mistral-7B")
mistral_model = AutoModelForCausalLM.from_pretrained("NousResearch/Hermes-2-Pro-Mistral-7B").to(device)
text_generator = pipeline("text-generation", model=mistral_model, tokenizer=mistral_tokenizer, device=0)

tokenizer_config.json:   0%|          | 0.00/6.82k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [56]:
# Extract relevant metadata from the queried matches
if response['matches']:
    best_match = response['matches'][0]
    #Nearest_match = response['matches'][1]
    #Nearest_metadata= Nearest_match['metadata']
    metadata = best_match['metadata']
    service_name = metadata['microservice']
    #textual_log = metadata['textual_representation']
    time1 = metadata['timestamp']
    availability_average = metadata['availability_Average']
    latency_average = metadata['latency_Average']
    latency_p50 = metadata['latency_p50']
    latency_p90 = metadata['latency_p90']
    latency_p95 = metadata['latency_p95']
    latency_p99 = metadata['latency_p99']
    requests_sum = metadata['requests_Sum']
    #time2 = Nearest_metadata['timestamp']
    mad_score = max_mad_score_row['Max_MAD_Score']  # Use MAD score from the highest anomaly row
    affected_metric = max_mad_score_row['Metric_With_Max_MAD']
    #dependencies = list(metadata.get('dependencies', []))
    #dependents = list(metadata.get('dependents', []))
else:
    raise ValueError("No matches found in the query response.")

# Get dependencies and dependents for the identified service
if service_name in service_info:
    dependencies = service_info[service_name]['dependencies']
    dependents = service_info[service_name]['dependents']
else:
    dependencies = []
    dependents = []

    dependencies_formatted = ", ".join(dependencies)
    dependents_formatted = ", ".join(dependents)

def preprocess_input(dependencies, dependents):
    dependencies_formatted = ', '.join(dependencies) if dependencies else 'No direct dependencies; focus on dependents or the anomaly itself.'
    dependents_formatted = ', '.join(dependents) if dependents else 'None'
    return dependencies_formatted, dependents_formatted

def clean_hypothesis(hypothesis):
    lines = hypothesis.split('\n')
    seen = set()
    unique_lines = [line for line in lines if line and line not in seen and not seen.add(line)]
    return '\n'.join(unique_lines)

def generate_analysis_prompt(service_name, mad_score, affected_metric, dependencies, dependents):
    dependencies_formatted, dependents_formatted = preprocess_input(dependencies, dependents)
    prompt = f"""
    An anomaly with a Median Absolute Deviation (MAD) score of {mad_score} has been detected in the {service_name} service's {affected_metric} metric, indicating a substantial deviation impacting its performance. This service is a critical component of a pet adoption website's microservices architecture.

    The service relies on the following dependencies: {dependencies_formatted}.
    The service also serves as a crucial dependency for: {dependents_formatted}.

    On {time1}, the service {service_name} reported metrics with an availability of {availability_average}%, an average latency of {latency_average} ms, and peak latencies at p50: {latency_p50}ms, p90: {latency_p90}ms, p95: {latency_p95}ms, and p99: {latency_p99} ms. Total requests were {requests_sum}. Use this data to support your analysis.

    Focus Areas:
    1. Root cause node and Target Node Identification:
        - What is the singular root cause node among the dependencies and dependents.
        - What is the primary target node among the dependents directly impacted by this anomaly
        - Your analysis should focus on identifying a singular root cause from among the dependencies and dependents.  Consider each dependency's role and potential issues that could lead to such a deviation.
        - Additionally, pinpoint the primary dependent (target node) that is most directly affected by this anomaly. This should be the service that relies on {service_name} and would face the most significant impact due to the anomaly in {affected_metric}. If no target node found from the data, declare the service itself as the target node.
    2. Dependencies and Their Impact:
        - Analyze the influence of {service_name} on its direct dependencies.
        - Assess how issues originating from {service_name} propagate to dependent services, affecting system performance and reliability.
    3. Pathways of Impact:
        - Map out the key pathways through which the issues are transmitted within the system.
    4. Metrics and Effects:
        - Evaluate how the issues affect critical performance metrics like latency and availability.
    5. Mitigation Strategies:
        - Propose actionable mitigation strategies to address the current issues.
        - Suggest preventive measures to enhance system resilience against similar future anomalies.
    Expected Outcomes:
        - Provide detailed insights into dependency-related impacts and propagation mechanisms.
        - Offer specific recommendations for both immediate resolution and long-term preventive strategies.
    Instructions:
        - Structure the response to ensure a logical flow, with each section addressing specific aspects as detailed above.
        - Highlight the importance of data-driven decision-making in managing microservice architectures.

    """
    print("Generated prompt:\n", prompt)
    return prompt.strip()

# Generate the prompt using the extracted metadata
prompt = generate_analysis_prompt(service_name, mad_score, affected_metric, dependencies, dependents)

# Generate the response using the Mistral model
response = text_generator(prompt, max_new_tokens=1500, num_return_sequences=1, temperature=0.7)[0]['generated_text']
cleaned_response = clean_hypothesis(response)

print("Generated Response:\n", cleaned_response)


Generated prompt:
 
    An anomaly with a Median Absolute Deviation (MAD) score of 108.22996195090894 has been detected in the petInfo_AWS::DynamoDB::Table service's requests_Sum metric, indicating a substantial deviation impacting its performance. This service is a critical component of a pet adoption website's microservices architecture.

    The service relies on the following dependencies: No direct dependencies; focus on dependents or the anomaly itself..
    The service also serves as a crucial dependency for: PetSearch_AWS::ECS::Fargate, payforadoption_AWS::ECS::Container, petlistadoptions_AWS::ECS::Fargate, lambdastatusupdater_AWS::Lambda::Function, PetSite.

    On 2023-04-22 06:50:00, the service petInfo_AWS::DynamoDB::Table reported metrics with an availability of 100.0%, an average latency of 0.0135342812254714 ms, and peak latencies at p50: 0.0038708019828941ms, p90: 0.0280731867769307ms, p95: 0.0459056850322154ms, and p99: 0.3736057591813309 ms. Total requests were 999.0.